# Scala notebook for analyzing archived IoT event data

This notebook uses Scala and Spark SQL to read and transform archived IoT event data. The purpose of the notebook is to analyze historic (i.e. archived) IoT Event Data comming from Elevators, and then determine at which dates specific elevators have been impacted by maintenance stops caused by overheated engines. 

The notebook has 3 sections:
1. Connect to Cloud Object Store to read the archived IoT event data into a Spark SQL Data Frame.
2. Filter and aggregate the event data so that it shows the dates where maintenance stops occured.
3. Save the information to a DB2 data table for further analysis e.g. by Business Intelligence tools.

Spark SQL is documented at: https://spark.apache.org/docs/latest/sql-programming-guide.html

## Import data from Cloud Object Storage

An initial step creates the connection to the Cloud Object Storage and reads the data from the archived Cloudant NoSQL Database into a Spark SQL Data Frame. A a second step prints out the schema of the Data Frame as well as a couple of rows of event data.

In [1]:
// Insert the connection to the file on Cloud Object Storage
<here>

[Stage 72:>                                                         (0 + 2) / 2]

In [2]:
// if this variable is true, the notebook will print out the intermediate results for debugging/testing purposes.
val debug = false
if (debug) { dfData1.show(5); dfData1.printSchema()}

# Filter and aggregate event data using Spark SQL

The initial dataframe will be filtered by removing empty rows as wells as event data where the motor temperature is below 200 degree Fahrenheit. In a final step, the information will be aggregated and sorted to show just the elevator name, the date as well as the max motor temperature on the date where an issue occured. 

In [3]:
// Filter out null rows
val dfData2 = dfData1.filter(dfData1.col("key").!==("null"))

In [4]:
// Select just the required set of columns
val dfData3 = dfData2.select("doc.deviceId","doc.timestamp","doc.data.d.motorTemp")
if (debug) {dfData3.show()}

In [5]:
// Search for rows where the motor temperature went above 200 degree Fahrenheit
val dfData4 = dfData3.filter(dfData1.col("doc.data.d.motorTemp").>(200))
if (debug) {dfData4.show()}

In [6]:
// Select the date from the timestamp and cast the resulting string to a proper date type
val dfData5 = dfData4.withColumn("date", dfData4.col("timestamp").substr(0,10).cast("date"))
dfData5.cache()
if (debug) {dfData5.printSchema(); dfData5.show()}

In [7]:
// Group by device id (elevator name) and date and determine the max temperatur for that date
val dfData6 = dfData5.groupBy("deviceId","date").max("motorTemp")

In [8]:
// Order by date
val dfData7 = dfData6.orderBy(dfData6.col("date").asc)
if (debug) {dfData7.printSchema(); dfData7.show()}

# Write to DB2

Writing to DB2 is done in this notebook using a simple scheme that will create the database table and simply append the rows in the data frame to the table.

The code has been inspired by the following two resources:
1. http://bigdatums.net/2016/10/16/writing-to-a-database-from-spark/
2. http://support.datascientistworkbench.com/knowledgebase/articles/829689-access-dashdb-or-db2-using-jdbc-from-scala-noteb

In [9]:
// Insert the credentials for the Db2 service 
<here>

In [10]:
// Compute properties and URL for the connection
val prop = new java.util.Properties
prop.setProperty("driver", "com.ibm.db2.jcc.DB2Driver")
prop.setProperty("user", credentials_1("user"))
prop.setProperty("password", credentials_1("password")) 
 
//jdbc mysql url - destination database is named "data"
//val url = "jdbc:mysql://localhost:3306/data"
val url = List("jdbc:db2://", credentials_1("host"), ":", credentials_1("port"), "/", credentials_1("database")).mkString("");
 
//destination database table 
val table = "sample_data_table"

In [11]:
//write data from spark dataframe to database
dfData7.write.mode("append").jdbc(url, table, prop)

[Stage 77:===================>                                      (1 + 2) / 3]

In [12]:
dfData7.show()

+----------+----------+--------------+                                          
|  deviceId|      date|max(motorTemp)|
+----------+----------+--------------+
|Elevator01|2017-08-03|         220.0|
|Elevator01|2017-08-07|         220.0|
|Elevator03|2017-08-07|         213.0|
+----------+----------+--------------+

